# Clustering

---

**intro to K-Means clustering**

Clustering falls into the category of "unsupervised learning," i.e. we use it to explore unlabeled data (or labeled data with the labels held-out) to extract information about the shape of the data. This places an extra burden on our workload, however, since we'll need to make several assumptions about our data as well as using methods of evaluation that aren't consistently agreed upon. As a data analyst / engineer / scientist, there is always an expectation of you to communicate effectively about data and your analysis of it. Visualizing data appropriately for your audience is a key part of this, and will be a serious focus in this notebook.

We'll keep this fairly elementary, but you can read more about [in the sklearn docs](https://scikit-learn.org/stable/modules/clustering.html#k-means) if you need to ramp up the difficulty.

In a nutshell, the goal of K-Means is to take a given number of points (called centroids) and move them around until they are at points of best fit for the given data. 
* the sum of squared distances from each point in a cluster to its centroid is minimized: $\sum_{i=0}^{n} \displaystyle \min_{\mu_j \in C}(||x_i - \mu_j||^2)$

The solution to this formula is also called inertia, and is one of the methods used to estimate the ideal number of clusters.

**Contents:**

0. Exploratory Data Analysis
1. Data Transformation
2. Determining K
3. Analyzing Results

---

### Exploratory Data Analysis

...often just EDA. We did this step in the last notebook, but I think it's important to mention that there is quite a range of expectation for EDA. Depending on the goal - personal use, reporting to an analytics manager, reporting to a CEO - you'll need to take a different approach to how you summarize the data that you're working with. If you're lucky, someone before you has already put in place some guidelines for this. If it's up to you, document extensively and get someone to proof-read it for you.

In [ ]:
# standard imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# let's take another popular dataset from uci for this exercise
# note

# source: columns copy-pasted from .name file

cols = ['Class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium',
        'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
        'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']

df = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data',
                 names=cols)
df.head()

In [ ]:
# the ranges of the features for this dataset are quite different, look at
# Proline compared to any other column

df.describe().round(2)

In [ ]:
# boxplots aren't always the best option. seaborn is a package that has a ton
# of visualization capabilities, one of which is the pairplot

# note, this one is a bit slow and can become impossible to interpret/run at higher
# dimensions, so be careful

import seaborn as sns

sns.pairplot(df.iloc[:,1:]);

---

### Data Transformation

We saw with df.describe that there is quite a bit of disparity bewteen the ranges of our data features. Algorithms like KNN and KMeans measure the distance between points, so without standardizing our data a small percentage change in 'Proline' will eclipse larger percentage changes in any of the other features. For certain datasets this may actually be advantageous, but since we're playing uncertain we'll need to remove this imbalance.

One method is to "normalize" our data - force it into the standard normal distribution N(0, 1). The formula to accomplish this should look familiar to someone with a stats background: $Z = \frac{(x - \mu)}{\sigma}$

But we can't start off by transforming all of our data (refer to lecture about not training on testing data). First we need to train-test split, then we can normalize our training data while preserving the same $\mu$ and $\sigma$ to transform our testing data.

In [ ]:
from sklearn.model_selection import train_test_split  # hello again
from sklearn import preprocessing  # contains classes for normalization/standardizing

seed = 42  # always

# for efficiency, I call X and y directly from the data
# recall df.iloc[row_start:row_end, column_start:column_end]
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:], df['Class'],
                                                    test_size = 0.2,
                                                    random_state = seed)

scaler = preprocessing.StandardScaler().fit(X_train)  # fit to training data

# transform both train and test with the fitted mu and sigma
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# now look at the ranges of Proline and Alcohol (bottom left of pairplots)
# note: a few (test) points are missing

plt.figure(figsize=(6, 6))
plt.scatter(X_train[:,0], X_train[:,-1])
plt.xlabel('Alcohol')
plt.ylabel('Proline');

---

### Determining K

The most popular method for determining K is "the elbow method" using inertia, the silhouette score, or both. 

We went over inertia earlier, and the silhouette score (or silhouette coefficient) isn't too far removed from that concept. To calculate it for a point, we take the mean distance between that point and all other points in its cluster ($a$) and the mean distance between that point and all other points in the nearest cluster ($b$). Then the formula $s = \frac{b - a}{\max(a, b)}$ gives us the coefficient for that point. Taking the mean of all silhouette coefficents gives the silhouette score.

One other method is using hierarchical clustering to determine the optimal number of starting centroids and their starting points.

In [ ]:
# clustering imports
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

cluster_counts = [2, 3, 4, 5, 6]  # number of centroids to try
silhouettes = []  # storage for scores
inertias = []

for count in cluster_counts:
    
    km = KMeans(n_clusters=count, random_state=seed)
    cluster_labels = km.fit_predict(X_train)  # fit_predict does exactly that, fit first then predict
    
    silhouettes.append(silhouette_score(X_train, cluster_labels))  # s-score
    inertias.append(km.score(X_train))  # inertia
    
# plot both scores per cluster on subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

ax1.plot(cluster_counts, silhouettes)
ax1.set_title('silhouette scores by cluster', fontweight='bold')
ax1.set_xlabel('number of clusters')
ax1.set_ylabel('silhouette scores')
ax1.set_xticks([2, 3, 4, 5, 6])

ax2.plot(cluster_counts, inertias)
ax2.set_title('inertia by cluster', fontweight='bold')
ax2.set_xlabel('number of clusters')
ax2.set_ylabel('inertia')
ax2.set_xticks([2, 3, 4, 5, 6])

plt.show()

# for silhouette score, we're looking for the global maximum score (3)
# for inertia, we're looking for the "elbow" where inertia stops increasing quickly (not so clear here)

In [ ]:
# read more here (https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering)
X = linkage(X_train, method='ward')

plt.figure(figsize=(16, 8))
dendrogram(X)  # specialized plot
plt.title('wine dendrogram', fontweight='bold')
plt.xlabel('individual observations')
plt.xticks([])  # don't display observation labels, it's crowded
plt.ylabel('cluster distances')
plt.grid(axis='y', ls='--')

plt.show()

# it's pretty clear that we have 3 clusters

In [ ]:
# we could simply stop here and initialize kmeans with 3 centroids
# but using the centroids from our ward clustering will vastly increase our efficiency
# - keep this method in mind if you need to cluster a large dataset

centroid_members = fcluster(X, t=3, criterion='maxclust')  # list of centroids by point
# if you want to verify, print centroid_members and 
# comment out plt.xticks([]) above to view the points

centroids = {}  # simple dict creation

for idx, i in enumerate(centroid_members):
    if i not in centroids.keys():
        centroids[i] = np.matrix(X_train[idx])  # create cluster_number: [indices]
    else:
        centroids[i] = np.vstack((centroids[i], X_train[idx]))  # append index to [indices]
        
centroid_points = [centroids[key].mean(0) for key in centroids.keys()]  # take means of each column

# reference first row of single row matricies, and convert to array of lists
centroid_points = np.asarray([i[0] for i in np.asarray(centroid_points)])
centroid_points  # ward centroids

In [ ]:
# now initialize kmeans with 3 clusters at the ward centroids
km = KMeans(n_clusters=3, random_state=seed, init=centroid_points)
km.fit(X_train);

km.cluster_centers_  # note how we've gone from 10 random inits to just 1 non-random

# compare the km centers to the ward centers

In [ ]:
# feel free to copy paste this in the future,
# it was a pain to make and should be easy to modify

def pca_plot(X, y, X_main, n_components=2):
    '''
    get pca plot of X and y, fit on X_main, with n_components
    
    change colors, patches, and legend according to data
    '''
    import matplotlib.pyplot as plt
    import matplotlib
    from sklearn.decomposition import PCA
    
    pca = PCA(n_components=n_components)
    pca.fit(X_main)
    pca_scores = pca.transform(X)

    color_mapper = {label: idx for idx, label in enumerate(set(y))}
    color_column = [color_mapper[label] for label in y]
    colors = ['orange', 'blue', 'red']

    plt.figure(figsize=(10, 10))    
    plt.scatter(pca_scores[:,0], pca_scores[:,1],
                s=8, alpha=.8,
                c=y,
                cmap=matplotlib.colors.ListedColormap(colors))
    orange_patch = matplotlib.patches.Patch(color='orange', label='cluster 1')
    blue_patch = matplotlib.patches.Patch(color='blue', label='cluster 2')
    red_patch = matplotlib.patches.Patch(color='red', label='cluster 3')
    plt.legend(handles=[orange_patch, blue_patch, red_patch], prop={'size': 10})

    plt.title('PCA plot of wine data')
    plt.xlabel('PCA dimension 1')
    plt.ylabel('PCA dimension 2')

    plt.show()
    
pca_plot(X_train, y_train, X_train)  # n_components=2 so we can get a flat plot

In [ ]:
# the clusters definitely aren't perfect, and if we wanted to increase performance
# we would go back to our data transformation step and do some dimensionality reduction
# to reduce the "curse of dimensionality"


# let's see how we do on the testing data though
pca_plot(X_test, y_test, X_train)